# Bike Sharing Demand Forecast

Data source: https://www.kaggle.com/c/bike-sharing-demand/data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# TODO: check & clean imports
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *

from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, r2_score, classification_report, mean_squared_error

In [ ]:
import itertools

## 1. Define Business Goal

Forecast bike demand given the data: datetime, season, holiday, workingday, weather, temp, atemp, humidity, windspeed.

Example in words: Given the forecasted weather conditions, how many bicycles can we expect to be rented out (city-wide) this Saturday at 2pm?

## 2. Get Data

In [ ]:
df = pd.read_csv("../data/bike-sharing-demand/train.csv")

In [ ]:
# datetime format string doc: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")

In [ ]:
df.info()

In [ ]:
df.set_index(keys='datetime', drop=False, inplace=True)

In [ ]:
df.isna().sum()

**count** is the sum of:
* **casual** - number of non-registered user rentals initiated
* **registered** - number of registered user rentals initiated

It belongs to our y data (target data), so we can omit it for now.

In [ ]:
df.drop(['casual', 'registered'], axis=1, inplace=True)

In [ ]:
df.head()

## 3. Train-Test-Split

Define X and y:

* X : Training data
* y : Target values

In [ ]:
df_full = pd.DataFrame(df) # keep a deep copy of unsplit data

In [ ]:
y = df['count']
X = df.drop(['count'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. Explore the Data

* **???** Explore on the *df* or the split data (problem of keeping track of y)
* **???** Explore time series *after* splitting? It will introduce random holes in equal sampled data.

In [ ]:
# merge for exploration
df_train = X_train.join(other=y_train)
#df = df_train

In [ ]:
df.head()

In [ ]:
df['count'].plot(figsize=(12,5))

The demand contains a positive trend over the years. Possible strategies may be applied:

* https://machinelearningmastery.com/time-series-trends-in-python/
* https://www.investopedia.com/terms/d/detrend.asp

In [ ]:
df.iloc[:1000]['count'].plot(figsize=(12,5), style='.-')

### Mind the data gaps

*\"For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month.\"*

### Include more data related to time and weather and moving average

In [ ]:
subdf = df.iloc[7000:8500]
window = '24H'

fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(ax=ax, data=subdf.rolling(window).mean(), x=df.index.name, y='count', color='k', legend='brief', alpha=0.7)
sns.scatterplot(ax=ax, data=subdf, x=df.index.name, y='count', hue='workingday', alpha=0.65)
ax.legend(['Avg ({})'.format(window), 'Working Day', 'No Working Day'])
ax

Comparing work to non-work days you can see all scenarios:
* more than local avg demand
* same ...
* less ...

In [ ]:
subdf = df.iloc[7000:8500]
window = '24H'

fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(ax=ax, data=subdf.rolling(window).mean(), x=df.index.name, y='count', color='k', legend='brief', alpha=0.7)
sns.scatterplot(ax=ax, data=subdf, x=df.index.name, y='count', hue='weather', alpha=0.65)
ax

Demand dips on days with bad weather.

Accessing DataFrames by datetime: https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

### Weather Type 4 almost not present in data

* 1: Clear, Few clouds, Partly cloudy, Partly cloudy
* 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
* 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
* 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

In [ ]:
df['weather'].plot.hist()

In [ ]:
df.loc[df['weather'] > 3]

### Explore correlations

In [ ]:
df.head()

In [ ]:
subdf = df.rolling('24H').mean()
ax = sns.scatterplot(       data=df   , y='count', x='temp', alpha=0.1)
ax = sns.scatterplot(ax=ax, data=subdf, y='count', x='temp', alpha=0.1)
ax

In [ ]:
feat = 'atemp'
subdf = df.rolling('24H').mean()
ax = sns.scatterplot(       data=df   , y='count', x=feat, alpha=0.1)
ax = sns.scatterplot(ax=ax, data=subdf, y='count', x=feat, alpha=0.1)
ax

Higher temps -> higher demand

In [ ]:
feat = 'humidity'
subdf = df.rolling('24H').mean()
ax = sns.scatterplot(       data=df   , y='count', x=feat, alpha=0.1)
ax = sns.scatterplot(ax=ax, data=subdf, y='count', x=feat, alpha=0.1)
ax

Humidity above 80 goes with a decrease in demand.

In [ ]:
feat = 'windspeed'
subdf = df.rolling('24H').mean()
ax = sns.scatterplot(       data=df   , y='count', x=feat, alpha=0.1)
ax = sns.scatterplot(ax=ax, data=subdf, y='count', x=feat, alpha=0.1)
ax

High windspeeds go with decrease in demand.

### Ideas

* Bin humidity/temps and calculate mean deamand for each range. (With help of sklearn preprocessors)
* temp vs atemp correlation: does it actually contain additional information?
* use season, hoiday, workday, wheathertype
* use resample method (props to catarina)
* use conditional probability/thinking: e.g. if wheather = 1 ==> stronger correlation between temp and demand

In [ ]:
# binning by temperature and plot mean demand
df['temp_b'] = pd.cut(x=df['temp'], bins=int((df['temp'].max()-df['temp'].min())/1.5))

In [ ]:
subdf=df[['temp_b','count']].groupby('temp_b').mean()
subdf['count'].plot(figsize=(18,5))

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
# thanks to alisa
sns.boxplot(ax=ax, data=df[['temp_b','count']], x='temp_b', y='count')
ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=90)
pass

In [ ]:
# binning by windspeed and plot mean demand
df['windspeed_b'] = pd.cut(x=df['windspeed'], bins=10)
subdf = df[['windspeed_b','count']].groupby('windspeed_b').mean()
subdf['count'].plot(figsize=(28,5))

## 5. Modelling
### 5.1 Feature engineering

#### Add time related features

* Day number since unix epoch (for implicit detrend of demand)
* Week day, hour, day of month  (one-hot-encoded and/or ordinal)

#### Additional features

* Interaction terms, Polynomial terms

#### Helpful material
* datetime docs: https://docs.python.org/3/library/datetime.html#datetime.datetime.timestamp
* datetime in pandas: https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587

In [ ]:
X_train.head()

#### Considerations

* Hour, dayofweek, month -> numeric or one-hot-encoded ?
* Wheather -> num and/or one-hot ?

#### Further ideas

* StandardScaler after polynomial/interaction expansion
* Bin and one-hot-encode continuous variables like temperature.
* Introduce new one-hot-encoded time variable: Quarter of the day.
* Remove features and look at the impact.
* Hyperparameter optimization and evaluation
    * Other regression types (Ridge, ElasticNet)
    * Use *GridSearchCV*

In [ ]:
def expand_to_hour(df):
    df.iloc[:,0] = df.iloc[:,0].dt.hour
    return df

def expand_to_dayofweek(df):
    df.iloc[:,0] = df.iloc[:,0].dt.dayofweek
    return df

def expand_to_month(df):
    df.iloc[:,0] = df.iloc[:,0].dt.month
    return df

def expand_to_quarter(df):
    df.iloc[:,0] = df.iloc[:,0].dt.quarter
    return df

def expand_to_year(df):
    df.iloc[:,0] = df.iloc[:,0].dt.year
    return df

def expand_to_timestamp(df, since="2000/01/01"):
    "Timestamp in days"
    timedelta = df.iloc[:,0] - pd.to_datetime(since)
    DAY_IN_SECONDS = 60*60*24
    df.iloc[:,0] = timedelta.dt.total_seconds()/(DAY_IN_SECONDS)
    return df

In [ ]:
time_features_numerical_encoded = []
for expand_func in [expand_to_hour, expand_to_dayofweek, expand_to_month
                    ,expand_to_year, expand_to_quarter, expand_to_timestamp]:
    item = make_pipeline(FunctionTransformer(func=expand_func)
                         ,StandardScaler()) , ['datetime']
    time_features_numerical_encoded.append(list(item))

In [ ]:
time_features_one_hot_encoded = [
    # Quarter of the day in one-hot-encoding
    [ make_pipeline(FunctionTransformer(func=expand_to_hour)
                    ,KBinsDiscretizer(n_bins=4, strategy='uniform'
                                       ,encode='onehot')), ['datetime'] ]
]
for expand_func in [expand_to_hour, expand_to_dayofweek, expand_to_month
                    ,expand_to_year, expand_to_quarter]:
    item = make_pipeline(FunctionTransformer(func=expand_func)
                         ,OneHotEncoder(sparse=False)) , ['datetime']
    time_features_one_hot_encoded.append(list(item))

In [ ]:
physical_continuous_features = [[ StandardScaler() , ['windspeed', 'humidity', 'temp', 'atemp'] ]]
day_category_features = [[ OneHotEncoder(sparse=False) , ['workingday', 'holiday'] ]]

wheather_feature_numerical = [[ StandardScaler() , ['weather'] ]]
wheather_feature_ohe = [[ OneHotEncoder(sparse=False) , ['weather'] ]]

season_feature_numerical = [[ StandardScaler() , ['season'] ]]
season_feature_ohe = [[ OneHotEncoder(sparse=False) , ['season'] ]]

In [ ]:
all_feature_sets = [
    time_features_numerical_encoded
    ,time_features_one_hot_encoded
    ,physical_continuous_features
    ,day_category_features
    ,wheather_feature_numerical
    ,wheather_feature_ohe
    ,season_feature_numerical
    ,season_feature_ohe
]
all_features = []
[all_features.extend(features) for features in all_feature_sets]
column_transformer = make_column_transformer(*all_features)

Todo: Try different combinations of `all_feature_sets` with the help of `itertools.combinations()`

In [ ]:
transformer = make_pipeline( column_transformer
                            ,PolynomialFeatures(degree=2) )
#transformer = column_transformer # override PolynomialFeatures
transformer

In [ ]:
dd = transformer.fit_transform(X_train)
#for i in range(30):
#    print(dd[i][1],dd[i][-4:])
dd.shape

In [ ]:
transformer.fit(X_train)
X_train_tf = transformer.transform(X_train)
X_test_tf  = transformer.transform(X_test)
X_train_tf.shape, X_test_tf.shape

### 5.2 Train model and evaluate

In [ ]:
elastic_net_m = ElasticNet()

In [ ]:
elastic_net_m.get_params()

In [ ]:
hyperparams = {
    'alpha': np.linspace(0,1,21),
    'l1_ratio': np.linspace(0,1,11)
}

In [ ]:
m = GridSearchCV(estimator=elastic_net_m, param_grid=hyperparams, cv=5, scoring='r2') 

In [ ]:
m.fit(X_train_tf, y_train)

Inspect coefficients

In [ ]:
_tol = 0.01
(m.coef_ < _tol).sum()#, m.coef_

In [ ]:
print("Show features that got selected by L1 regularization (# are non-zero coeffs):")
for coef in m.coef_:
    print("#" if coef > _tol else "_", end=' ')

In [ ]:
y_pred_test = m.predict(X_test_tf)
y_pred_train = m.predict(X_train_tf)

In [ ]:
y_pred_test
sns.histplot(data=pd.DataFrame({'y_pred_test':y_pred_test, 'y_test':y_test}))

Distribution looks fine. Negative demand has to be clamped to zero in the post.

In [ ]:
m.score(X_train_tf, y_train)

In [ ]:
r2_score(y_train, y_pred_train)

In [ ]:
r2_score(y_test, y_pred_test)

In [ ]:
print('MSE:'
      , mean_squared_error(y_train, m.predict(X_train_tf))
      , mean_squared_error(y_test, m.predict(X_test_tf)))

In [ ]:
scores = cross_val_score(m, X_train_tf, y_train, cv=5, scoring='r2')

In [ ]:
scores.mean(), scores

No signs of overfitting so far.

## 6. Second approach: Hyperparameter Optimization
### 6.1 Feature engineering